In [2]:
pip install pysam


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 18.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pysam
from collections import defaultdict
import re
import os

In [25]:
# --- Input files ---
bam_files = {
    "Pro": "/Volumes/projects-t3/SerreDLab-3/fdumetz/Leishmania/featurecounts/Ld1S_3ONT_SL+polyA_monocistron.bam",
    "Ama": "/Volumes/projects-t3/SerreDLab-3/fdumetz/Leishmania/featurecounts/Ld1S_Ama_monocistron_SL.bam",
    "All": "/Volumes/projects-t3/SerreDLab-3/fdumetz/Leishmania/Ld_annotation/Fuzznuc/Ld_3ONT_asmfinal.bam"
}
gff3_file = "/Volumes/projects-t3/SerreDLab-3/fdumetz/Leishmania/featurecounts/Ld1S_pre-final4.gff3"
output_file = "/Volumes/projects-t3/SerreDLab-3/fdumetz/Leishmania/featurecounts/transcript_counts.txt"

In [26]:
# --- Parse GFF3 ---
print("Parsing GFF3...")
transcript_exons = defaultdict(list)

with open(gff3_file, 'r') as gff:
    for line in gff:
        if line.startswith("#"):
            continue
        fields = line.strip().split("\t")
        if len(fields) < 9 or fields[2].lower() != "exon":
            continue
        chrom, _, _, start, end, _, strand, _, attr = fields
        start, end = int(start), int(end)

        tid_match = re.search(r'transcript_id[ ="]*([^";\s]+)', attr)
        if not tid_match:
            tid_match = re.search(r'Parent=([^";\s]+)', attr)
        if not tid_match:
            tid_match = re.search(r'ID=([^";\s]+)', attr)
        if tid_match:
            tid = tid_match.group(1)
            transcript_exons[tid].append((chrom, start, end))

print(f"Found {len(transcript_exons)} transcripts.")

# --- Count reads per transcript per BAM ---
all_counts = defaultdict(lambda: defaultdict(int))

for sample_name, bam_path in bam_files.items():
    print(f"Processing {sample_name}...")

    bam = pysam.AlignmentFile(bam_path, "rb")
    for tid, exons in transcript_exons.items():
        seen_reads = set()
        for chrom, start, end in exons:
            try:
                for read in bam.fetch(chrom, start - 1, end):
                    if read.is_unmapped:
                        continue
                    if read.query_name not in seen_reads:
                        if read.reference_start < end and read.reference_end > start:
                            all_counts[tid][sample_name] += 1
                            seen_reads.add(read.query_name)
            except ValueError:
                continue
    bam.close()

# --- Write output matrix ---
print("Writing output...")
sample_names = list(bam_files.keys())
with open(output_file, "w") as out:
    out.write("transcript_id\t" + "\t".join(sample_names) + "\n")
    for tid in sorted(transcript_exons):
        counts = [str(all_counts[tid][s]) for s in sample_names]
        out.write(f"{tid}\t" + "\t".join(counts) + "\n")

print(f"Done. Output written to {output_file}")

Parsing GFF3...
Found 8466 transcripts.
Processing Pro...
Processing Ama...
Processing All...
Writing output...
Done. Output written to /Volumes/projects-t3/SerreDLab-3/fdumetz/Leishmania/featurecounts/transcript_counts.txt
